In [10]:
import torch
from torch.utils.data import ConcatDataset
from typing import List
from model import FreckerDataSet

def load_frecker_datasets(dataset_base_dir: str, iter_now: int, training_dataset_cross: int) -> ConcatDataset:
    """
    加载并合并多个Frecker数据集文件
    
    Args:
        dataset_base_dir (str): 数据集基础目录
        iter_now (int): 当前迭代次数
        training_dataset_cross (int): 跨数据集训练的数量
        
    Returns:
        ConcatDataset: 合并后的数据集
    """
    datafiles = []
    if iter_now == 0:
        datafiles.append(f"{dataset_base_dir}/1.h5")
    else:
        for i in range(
            max(1, iter_now - training_dataset_cross),
            iter_now + 2
        ):
            datafiles.append(f"{dataset_base_dir}/{i}.h5")
    
    datasets = [FreckerDataSet(x) for x in datafiles]
    dataset = ConcatDataset(datasets)
    print(f"加载了 {len(dataset)} 个样本")
    
    return dataset

In [14]:

# 加载数据集
dataset = load_frecker_datasets(
    dataset_base_dir="/mnt/cdata/data-1/",
    iter_now=142,
    training_dataset_cross=20
)

# 之后可以根据需要自行分割数据集
train_size = int(0.8 * len(dataset))
train_dataset = torch.utils.data.Subset(dataset, range(train_size))
val_dataset = torch.utils.data.Subset(dataset, range(train_size, len(dataset)))

加载了 361309 个样本
